In [2]:
from langchain.document_loaders import PyPDFLoader

In [3]:
import dotenv
import os
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from langchain_groq import ChatGroq

In [17]:
api_key=os.getenv("GROQ_API")
llm=ChatGroq(groq_api_key=api_key,model="llama-3.3-70b-versatile")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001FBD1CA17E0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001FBD1CA2410>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain.schema import SystemMessage,HumanMessage,AIMessage

### STUFF_DOCUMENT_CHAIN

In [7]:
from langchain.chains import LLMChain
from langchain import PromptTemplate

In [10]:
loader = PyPDFLoader('Comparative_Analysis_of_Machine_Learning.pdf')
f = loader.load_and_split()

In [9]:
!pip install pypdf

In [11]:
f

[Document(metadata={'producer': 'GPL Ghostscript 9.26', 'creator': '', 'creationdate': "D:20231110215223Z00'00'", 'moddate': "D:20231110215223Z00'00'", 'title': '', 'author': '', 'subject': '', 'keywords': '', 'source': 'Comparative_Analysis_of_Machine_Learning.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}, page_content='Comparative Analysis of Machine Learning Algorithms in Predicting Rate of \nPenetration during Drilling\nOlaosebikan Abidoye Olafadehan*, Ikenna David Ahaotu\nDepartment of Chemical and Petroleum Engineering, University of Lagos, Akoka-Y aba, Lagos 101017, Nigeria\nCitation: Olafadehan OA, Ahaotu  ID, Comparative Analysis of Machine Learning Algorithms in Predicting Rate of Penetration \nduring Drilling. J Petro Chem Eng 2023;1(1): 32-47.\nReceived: 14 October, 2023; Accepted: 31 October, 2023; Published: 07 November, 2023\n*Corresponding author: Olafadehan OA, Department of Chemical and Petroleum Engineering, University of Lagos, Akoka-\nY aba, Lagos 101017 N

In [26]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
f_o = RecursiveCharacterTextSplitter(chunk_size = 2000,chunk_overlap=400).split_documents(f)

In [27]:
f_o

[Document(metadata={'producer': 'GPL Ghostscript 9.26', 'creator': '', 'creationdate': "D:20231110215223Z00'00'", 'moddate': "D:20231110215223Z00'00'", 'title': '', 'author': '', 'subject': '', 'keywords': '', 'source': 'Comparative_Analysis_of_Machine_Learning.pdf', 'total_pages': 16, 'page': 0, 'page_label': '1'}, page_content='Comparative Analysis of Machine Learning Algorithms in Predicting Rate of \nPenetration during Drilling\nOlaosebikan Abidoye Olafadehan*, Ikenna David Ahaotu\nDepartment of Chemical and Petroleum Engineering, University of Lagos, Akoka-Y aba, Lagos 101017, Nigeria\nCitation: Olafadehan OA, Ahaotu  ID, Comparative Analysis of Machine Learning Algorithms in Predicting Rate of Penetration \nduring Drilling. J Petro Chem Eng 2023;1(1): 32-47.\nReceived: 14 October, 2023; Accepted: 31 October, 2023; Published: 07 November, 2023\n*Corresponding author: Olafadehan OA, Department of Chemical and Petroleum Engineering, University of Lagos, Akoka-\nY aba, Lagos 101017 N

In [28]:
template=""" Write a concise and detailed summary of the following texts. Make sure to capture the technical terms in it. its very important,
text :{text}

 """
 
from langchain.prompts import PromptTemplate
prompt = PromptTemplate(template=template,input_variables=["text"])

In [29]:
final_template = """
Provide the final summary of the entire write-up with the important points, that are tailored towards the title of the doucment.
{text} 
"""

final_prompt_template=PromptTemplate(input_variables=['text'],template=final_template)
final_prompt_template

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide the final summary of the entire write-up with the important points, that are tailored towards the title of the doucment.\n{text} \n')

In [30]:
prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template=' Write a concise and detailed summary of the following texts. Make sure to capture the technical terms in it. its very important,\ntext :{text}\n\n ')

In [31]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm=llm,map_prompt=prompt,chain_type='map_reduce',verbose=True,combine_prompt=final_prompt_template)

In [32]:
output = chain.run(f)



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 Write a concise and detailed summary of the following texts. Make sure to capture the technical terms in it. its very important,
text :Comparative Analysis of Machine Learning Algorithms in Predicting Rate of 
Penetration during Drilling
Olaosebikan Abidoye Olafadehan*, Ikenna David Ahaotu
Department of Chemical and Petroleum Engineering, University of Lagos, Akoka-Y aba, Lagos 101017, Nigeria
Citation: Olafadehan OA, Ahaotu  ID, Comparative Analysis of Machine Learning Algorithms in Predicting Rate of Penetration 
during Drilling. J Petro Chem Eng 2023;1(1): 32-47.
Received: 14 October, 2023; Accepted: 31 October, 2023; Published: 07 November, 2023
*Corresponding author: Olafadehan OA, Department of Chemical and Petroleum Engineering, University of Lagos, Akoka-
Y aba, Lagos 101017 Nigeria, Phone: +234802-912-9559, Email: oolafadehan@unilag.edu.ng
Copyright: © 2023 Olafadehan

c:\Users\NEW USER\anaconda3\envs\genv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\NEW USER\anaconda3\envs\genv\lib\site-packages\huggingface_hub\file_download.py:142: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\NEW USER\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode,



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Provide the final summary of the entire write-up with the important points, that are tailored towards the title of the doucment.
Here is a concise and detailed summary of the text, capturing the technical terms:

This study presents a comparative analysis of machine learning (ML) algorithms in predicting the rate of penetration (ROP) during drilling in the petroleum industry. The authors evaluated the performance of seven ML algorithms: Support Vector Machine Regression (SVR), Random Forest Regression (RF), Linear Regression (LR), K-Nearest Neighbors (KNN), Stacking technique, Voting technique, and Convolutional Neural Network (CNN). The data used for this investigation was obtained from an oil well in Nigeria and was split into training and testing sets in a 60:40 ratio. The results showed that the Stacking technique had the best performance, with an accuracy of 99.8% and 97.5% o

In [33]:
print(output)

**Final Summary: Prediction and Optimization of Rate of Penetration (ROP) in Drilling Operations**

This research summary provides an overview of the application of Machine Learning (ML) algorithms in predicting the Rate of Penetration (ROP) during drilling operations in the petroleum industry. The key findings and important points are:

* **ML algorithms can accurately predict ROP**: The study demonstrates the effectiveness of ML algorithms, such as Support Vector Machine Regression (SVR), Random Forest Regression (RF), Linear Regression (LR), K-Nearest Neighbors (KNN), Stacking technique, Voting technique, and Convolutional Neural Network (CNN), in predicting ROP with high accuracy.
* **Importance of drilling parameters**: The study highlights the importance of drilling parameters, such as weight on bit (WOB), rotary speed (RPM), and mud properties, in predicting ROP.
* **Data-driven approach**: The study uses a data-driven approach to estimate ROP, which overcomes the limitations of